In [2]:
import cProfile
import pstats

import subprocess
from memory_profiler import memory_usage

from cubes import get_cube

In [ ]:
print("|  N  |     DOFs |  FWD Time |  BWD Time |   Peak RAM |")
print("| --- | -------- | --------- | --------- | ---------- |")

device = "cpu"
for N in [10, 20, 30, 40, 50, 60, 70, 80, 90]:
    mem_usage, result = memory_usage(
        lambda: subprocess.run(
            ["python", "cubes.py", "-N", str(N), "-device", device],
            capture_output=True,
            text=True,
        ),
        retval=True,
        include_children=True,
        max_usage=True,
        interval=0.1,
    )

    print(result.stdout.strip() + f" {mem_usage:8.1f}MB |")

|  N  |     DOFs |  FWD Time |  BWD Time |        RAM |
| --- | -------- | --------- | --------- | ---------- |
|  10 |     3000 |     0.12s |     0.06s |    618.5MB |
|  20 |    24000 |     0.54s |     0.34s |    934.7MB |
|  30 |    81000 |     1.90s |     1.33s |   1949.0MB |
|  40 |   192000 |     4.84s |     4.00s |   4061.1MB |
|  50 |   375000 |    10.43s |     9.58s |   6877.9MB |
|  60 |   648000 |    19.05s |    21.73s |  11318.2MB |
|  70 |  1029000 |    34.34s |    40.30s |  16345.3MB |
|  80 |  1536000 |    55.64s |    56.67s |  23191.6MB |
|  90 |  2187000 |    82.74s |   136.69s |  32951.2MB |


### Memory and speed analysis for cube of size 70

In [3]:
# add @profile decorator to function to be profiled
cube = get_cube(70)
solution = cube.solve()

In [4]:
cProfile
cProfile.run("get_cube(70).solve()", "stats")
p = pstats.Stats("stats")
p.sort_stats("tottime").print_stats(10)

Fri Nov  1 10:10:50 2024    stats

         32487 function calls (32447 primitive calls) in 34.591 seconds

   Ordered by: internal time
   List reduced from 367 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      319   15.273    0.048   15.273    0.048 {built-in method scipy.sparse._sparsetools.csr_matvec}
        1    7.530    7.530    7.530    7.530 {method 'coalesce' of 'torch._C.TensorBase' objects}
        1    4.383    4.383   19.310   19.310 c:\Users\meyernil\AppData\Local\miniforge3\envs\torchfem\Lib\site-packages\scipy\sparse\linalg\_isolve\minres.py:10(minres)
        2    1.602    0.801    3.373    1.687 C:\Users\meyernil\torch-fem\src\torchfem\base.py:75(integrate)
       96    1.186    0.012    1.186    0.012 {built-in method torch.einsum}
       10    0.882    0.088   32.002    3.200 c:\Users\meyernil\AppData\Local\miniforge3\envs\torchfem\Lib\selectors.py:313(_select)
      198    0.744    0.004    0.744    0.004 